In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd "/content/drive/MyDrive"

/content/drive/MyDrive


# Libraries

In [52]:
# libraries
import nltk #NLP lib
import math
import numpy as np # numpy for arrays
import pandas as pd # pandas lib for data handling
from nltk.corpus import stopwords # common eng sstopwords
from nltk.stem import PorterStemmer #porter stemmer algo
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup # HTML to txt
import os
import sys
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
ps = PorterStemmer()
sw_nltk = stopwords.words('english') # bag of all common english stop words
import gensim # for LDA
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

[nltk_data] Downloading package punkt to /home/ksv/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ksv/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ksv/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ksv/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


## Supporting Functions

In [53]:
import re
def TokStem(text,j): # Tokenise and Stem the given text
    stoplist = set(nltk.corpus.stopwords.words('english'))
    # text = [contraction[word.islower()] for word in text.split(" ") if word.islower() in contractions.keys() ]
    # print(text)
    
    text =  nltk.word_tokenize(text.lower())
    # text = re.split(',text)    
    # print(text)
    
    words = text #[word for word in text if word.isalpha()]
    ltz = WordNetLemmatizer()
    # words = [word for word in text.split() if word.lower() not in sw_nltk] # bag of tokenised words
    final1 = []
    final = []
    punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~``'''
    contract_strings = ["'s", "\"","'ve"]
    for word in words:
      if word not in stoplist and word not in punc and word not in contract_strings and word!='"':
        
        final1.append(ltz.lemmatize(word))
        final.append(ps.stem(word))
    # print(final2)
    if j==1: # give a string
        return " ".join(final)
    return final
text = "When I C++ . first-day \"met! 'her' will her's not she_was very quiet. She remained quiet during the entire two hour long journey from Stony Brook to New York."
x = TokStem(text,0)
print(x)

['c++', 'first-day', 'met', "'her", 'she_wa', 'quiet', 'remain', 'quiet', 'entir', 'two', 'hour', 'long', 'journey', 'stoni', 'brook', 'new', 'york']


In [54]:
def TokStem(text,j): # Tokenise and Stem the given text
    words = [word for word in text.split() if word.lower() not in sw_nltk] # bag of tokenised words
    final = []
    for w in words:
        final.append(ps.stem(w))
    if j==1: # give a string
        return " ".join(final)
    return final
text = "When I first met her she was very quiet. She remained quiet during the entire two hour long journey from Stony Brook to New York."
x = TokStem(text,0)

In [55]:
def hbt(text): # convert HTML body to text
    x = BeautifulSoup(text).get_text()
    # print(x)
    return x

In [6]:
def rmTags(text): # convert tags to text
    st,out = [],[]
    for i in text:
        if i == '<':
            st = []
        elif i == '>':
            st += ' '
            out += st
        else:
            st += i
    return "".join(out[:-1]) # remove the last space

In [10]:
############ test code
x = "<xz><y>"
x += "<w>"
print(x)
y = rmTags(x)
print(y,len(y))
print(x[:-1])

<xz><y><w>
xz y w 6
<xz><y><w


## Loading the data 

In [7]:
df = pd.read_csv('Dataset.csv')
df

,RelatedPostId,OId,OTitle,OTags,OBody,DId,DTitle,DTags,DBody
0,4,4,How to convert a Decimal to a Double in C#?,<c#><floating-point><type-conversion><double><...,<p>I want to use a <code>Track-Bar</code> to c...,51027658,How to echo a JS variable to php?,<javascript><php>,<p>Is that possible to pass a JS variable to P...
1,9,9,How do I calculate someone's age based on a Da...,<c#><.net><datetime>,<p>Given a <code>DateTime</code> representing ...,2194999,How to calculate an age based on a birthday,<c#><asp.net-mvc><date-arithmetic>,<blockquote>\n<p><strong>Possible Duplicate:</...
2,11,11,Calculate relative time in C#,<c#><datetime><time><datediff><relative-time-s...,<p>Given a specific <code>DateTime</code> valu...,7392566,"How to get datetime in words like ""today"", ""ye...",<php><zend-framework><datetime><datetime-format>,<blockquote>\n <p><strong>Possible Duplicate:...
3,13,13,Determine a user's timezone,<html><browser><timezone><user-agent><timezone...,<p>Is there a standard way for a web server to...,4746249,get user timezone,<php><javascript><timezone>,<blockquote>\n <p><strong>Possible Duplicate:...
4,14,14,Difference between Math.Floor() and Math.Trunc...,<.net><math>,"<p>What is the difference between <a href=""htt...",43174619,What is the difference in floor function and t...,<math>,<p>What is the difference between Math.Floor()...
...,...,...,...,...,...,...,...,...,...
7880,479799,479799,Replace host in Uri,<c#><.net><uri>,<p>What is the nicest way of replacing the hos...,49486804,using regex to find and replace hostname in url,<c#><regex>,<p>Currently I have my program able to achieve...
7881,479891,479891,What does the Call keyword do in VB6?,<vba><vb6><syntax>,<p>There's some code in our project that looks...,50789914,Calling Macro from inside Macro - Error Compil...,<vba><ms-office><ms-word><word-2013>,<p>I have been copying code moslty to work on ...
7882,479897,479897,How to remove duplicates from Python list and ...,<python><list><sorting><unique>,"<p>Given a list of strings, I want to sort it ...",32706281,Fastest way to remove duplicates from a list o...,<python><list><duplicates>,<p>How can I remove duplicate items from a lis...
7883,479923,479923,Is C# a single dispatch or multiple dispatch l...,<c#><programming-languages><multiple-dispatch>...,<p>I'm trying to understand what single and mu...,58463016,Why always this code will call the object para...,<c#><overloading>,<p>Why the below code would call the <code>pub...


# Data Preprocessing

In [8]:
N1 = len(df)
N2 = 300
# duplicate questions for training (with sorting)
Q2 = df[['OId','OTitle','OBody','OTags']].loc[0:N1-1].copy() # original qns
Q1 = df[['DId','DTitle','DBody','DTags']].loc[0:N2-1].copy() # doop qns
print("Duplicate  question set")
Q1


Duplicate  question set


,DId,DTitle,DBody,DTags
0,51027658,How to echo a JS variable to php?,<p>Is that possible to pass a JS variable to P...,<javascript><php>
1,2194999,How to calculate an age based on a birthday,<blockquote>\n<p><strong>Possible Duplicate:</...,<c#><asp.net-mvc><date-arithmetic>
2,7392566,"How to get datetime in words like ""today"", ""ye...",<blockquote>\n <p><strong>Possible Duplicate:...,<php><zend-framework><datetime><datetime-format>
3,4746249,get user timezone,<blockquote>\n <p><strong>Possible Duplicate:...,<php><javascript><timezone>
4,43174619,What is the difference in floor function and t...,<p>What is the difference between Math.Floor()...,<math>
...,...,...,...,...
295,3582464,Is there a XSD-driven random XML test data gen...,"<p>For stress tests, I would like to create XM...",<c#><java><xml><delphi><xsd>
296,14279410,Using List<T> vs. IList<T>,<blockquote>\n <p><strong>Possible Duplicate:...,<c#><asp.net><.net>
297,2195747,Code to create a password encrypted zip file?,<p>What is the Python code to create a passwor...,<python>
298,7932008,Can float values add to a sum of zero?,<blockquote>\n <p><strong>Possible Duplicate:...,<c++><c><floating-point>


In [9]:

# # process the body wit tok, stem and modify tags and export

for i in range(Q2.shape[0]):
    
    x = TokStem(hbt(Q2.loc[i,'OBody']),1)
    y = rmTags(Q2.loc[i,'OTags'])
    z = TokStem(hbt(Q2.loc[i,'OTitle']),1)

    Q2.loc[i,'OBody'] = x
    Q2.loc[i,'OTags'] = y
    Q2.loc[i,'OTitle'] =z
  
for i in range(Q1.shape[0]):
    x = TokStem(hbt(Q1.loc[i,'DBody']),1)
    y = rmTags(Q1.loc[i,'DTags'])
    z = TokStem(hbt(Q1.loc[i,'DTitle']),1)
    
    Q1.loc[i,'DBody'] = x
    Q1.loc[i,'DTags'] = y
    Q1.loc[i,'DTitle'] =z

# Q2.to_csv('body_stem.csv') # export to CSV

## LDA processing

In [10]:
# also create master data for finding topics
data_words = []

for i in range(Q2.shape[0]):
  data_words.append((Q2.loc[i,'OBody']+Q2.loc[i,'OTitle']).split())

# create dictionary
id2word = corpora.Dictionary(data_words)

corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)
# LDA imp    
lda_model = gensim.models.LdaMulticore(corpus=corpus, id2word=id2word, num_topics=100,minimum_probability=0.0)
lda_train = np.zeros((len(data_words),100)) # Q1 + Q2 size by 100 topics, refer to perplexity in paper

for i in range(len(data_words)): #find prob distribution of qn belonging to all the topics
	lda_train[i] = np.array(lda_model[corpus[i]])[:,1]

# new = id2word.doc2bow(text)
# a = np.array(lda_model[text])[:,1]
# b = 

## Cosine Similarity between two strings

In [11]:
def cos_sim(a,b): # finding cosine similarity score for given 2 strings
    # input two strings that are split into words
    us = [] # creating union set
    us += b
    for x in a:
        if x not in b:
            us.append(x)
    freqa,freqb,frequ = dict(),dict(),dict()
    wa,wb = [], []
    for word in a: # freq bag for string a
        if word in freqa:
            freqa[word] += 1
        else:
            freqa[word] = 1
    for word in b:# freq bag for string b
        if word in freqb:
            freqb[word] += 1
        else:
            freqb[word] = 1
    for word in us: # freq bag for union of a and b
        if word in frequ:
            frequ[word] += 1
        else:
            frequ[word] = 1
    for i in range(len(us)): # calc TitleVec for a and b
        x = us[i]
        if x in a:
            wa.append(freqa[x]/frequ[x])
        else:
            wa.append(0)
        if x in b:
            wb.append(freqb[x]/frequ[x])
        else:
            wb.append(0)
    wa = np.array(wa)
    wb = np.array(wb)
    ans = np.dot(wa,wb)/(np.linalg.norm(wa)*np.linalg.norm(wb))
    if np.isnan(ans):
      return 0
    return  ans# cosine value
print(cos_sim(Q2.loc[0,'OTags'].split(),Q1.loc[0,'DTags'].split()))

0.0


# Calculating Similarity component scores

In [ ]:
#Run this if you want to retrain the training data
# b= ['OTitle','OBody','OTags']
# a = ['DTitle','DBody','DTags']
# SC = np.zeros([Q2.shape[0],Q1.shape[0],4]) # store all the component scores [ori,doop,4]
# y = 162130


# for i in range(Q2.shape[0]):
#     l11  = Q2.loc[i,b[0]].split()
#     l12  = Q2.loc[i,b[1]].split()
#     l13  = Q2.loc[i,b[2]].split()
#     for j in range(Q1.shape[0]):
#         l21  = Q2.loc[i,a[0]].split()
#         l22  = Q2.loc[i,a[1]].split()
#         l23  = Q2.loc[i,a[2]].split()

#         SC[i][j][0] = cos_sim(l11,l21) # TitleSC
#         SC[i][j][1] = cos_sim(l12,l22) # BodySC
#         SC[i][j][2] = cos_sim(Q2.loc[i,b[2]],Q1.loc[j,a[2]]) # TagsSC
b= ['OTitle','OBody','OTags']
a = ['DTitle','DBody','DTags']
SC = np.zeros([Q2.shape[0],Q1.shape[0],4]) # store all the component scores [ori,doop,4]
y = 162130
n2 = Q1.shape[0]
wa = np.zeros(100)
wb = np.zeros(100)
for i in range(Q2.shape[0]):
    print(i)
    for j in range(Q1.shape[0]):
        
        SC[i][j][0] = cos_sim(Q1.loc[j,a[0]].split(),Q2.loc[i,b[0]].split()) # TitleSC
        SC[i][j][1] = cos_sim(Q1.loc[j,a[1]].split(),Q2.loc[i,b[1]].split()) # BodySC
        SC[i][j][2] = cos_sim(Q1.loc[j,a[2]].split(),Q2.loc[i,b[2]].split()) # TagsSC
        text = (Q1.loc[j,'DBody']+Q1.loc[j,'DTitle']).split()
        corp = id2word.doc2bow(text)
        wa, wb = np.array(lda_model[corp])[:,1], lda_train[i] # lda train has Q1 qns and then Q2 qns
        SC[i][j][3] = np.dot(wa,wb)/(np.linalg.norm(wa)*np.linalg.norm(wb))# TopicSC
        if (np.isnan(SC[i][j][3])):
          SC[i][j][3] = 0
        if (j==35):
          print(SC[i][j])
np.save("Training set Similarity scores.npy",SC)


# Estimating parameters

In [12]:
SC = np.load("Training set Similarity scores.npy")
# instead of training everytime, we loaded the trained values into a file to skip training everytime we login

## Greedy Approach

In [ ]:
sp = np.linspace(0,1,2) # change last parameter for fine tuning
gco = 0 # global count

ga,gb,gc,gd = 0,0,0,0 # final parameter values
for a in sp:
    for b in sp:
        for c in sp:
            for d in sp:

                if a==0 and b==0 and c==0 and d==0:
                    continue
                tc = 0 # temp count
                for i in range(Q1.shape[0]): # iterate all duplicate qns
                    te = []
                    for j in range(Q2.shape[0]): # iterate over available qns
                        te.append(a*SC[j][i][0]+b*SC[j][i][1]+c*SC[j][i][2]+d*SC[j][i][3]) # composer score
                    x = te.index(max(te)) # get index of maximum element
                    if Q2.loc[x,'OId'] == df.loc[i,'OId']: # if correct original qn is detected
                        tc += 1
                if tc >= gco: # optimal value found
                    ga,gb,gc,gd = a,b,c,d
                    gco = tc
                    print(ga,gb,gc,gd,tc)
#             print(tc)
            
# print(SC[0][0:2])
# x = Q1.loc[0,a[1]]
# y = Q2.loc[0,b[0]]
# print(x,y)
# print(cos_sim(x,y))
# print(Q1.loc[0],Q1.loc[1])
# print(Q2.loc[0])

0.0 0.0 0.0 1.0 1
0.0 0.0 1.0 0.0 1


KeyboardInterrupt: ignored

## Training set 

Train paramerters for training data using  pseudo-greedy algo

In [ ]:
sp = np.linspace(0,1,2) # change last parameter for fine tuning
gco = 0 # global count
ga,gb,gc,gd = 0,0,0,0 # final parameter values
import random
k =  20

random_composers = np.zeros((100,5)) # store composer values, and recall rate
for iter in range(0,100):
  a = random.uniform(0,1)
  b = random.uniform(0,1)
  c = random.uniform(0,1)
  d = random.uniform(0,1)
  if a==0 and b==0 and c==0 and d ==0:
      continue
  tc = 0 # temp count
  count = 0
  de = {}
  for i in range(Q1.shape[0]): # iterate all duplicate qns
    te = []
    for j in range(Q2.shape[0]): # iterate over available qns
      te.append(a*SC[j][i][0]+b*SC[j][i][1]+c*SC[j][i][2] +d*SC[j][i][3]) # composer score
      de[j] = te[j]

    x = te.index(max(te)) # get index of maximum element
    # if Q2.loc[x,'OId'] == df.loc[i,'OId']: # if correct original qn is detected
    #   tc += 1

    temp_count= 0
    for key, val in sorted(de.items(), key= lambda item: item[1], reverse=True):
      
      if (temp_count<k):
        if Q2.loc[key,'OId'] == df.loc[i,'OId']:
          count+=1
          break
        temp_count+=1
      else:
        break
    
  random_composers[iter] = [a,b,c,d,count]
  print(count)
  if count >= gco: # optimal value found
    ga,gb,gc,gd = a,b,c,d
    gco = count
    print(ga,gb,gc,gd,count)
#             print(tc)

np.save("composer_score_recall.npy",random_composers)

In [ ]:
arr = np.loadtxt("composer_score_recall_float.npy")
print(arr)
# np.savetxt("composer_score_recall_float.npy",arr, fmt='%f')

[[7.66942e-01 5.58142e-01 6.18187e-01 6.34700e-01 1.99000e+02]
 [2.17466e-01 1.05721e-01 1.32979e-01 2.32762e-01 1.62000e+02]
 [9.51983e-01 2.02783e-01 1.82110e-01 6.85130e-01 1.63000e+02]
 [7.38470e-01 3.30363e-01 6.36086e-01 3.34402e-01 2.11000e+02]
 [1.56423e-01 3.74802e-01 5.38216e-01 6.18922e-01 1.42000e+02]
 [6.82558e-01 6.49785e-01 1.05780e-01 6.58650e-02 2.20000e+02]
 [1.58002e-01 1.15691e-01 4.67325e-01 7.79302e-01 1.06000e+02]
 [5.53511e-01 3.03638e-01 9.15385e-01 4.70785e-01 1.92000e+02]
 [5.42153e-01 3.85910e-02 2.75130e-01 8.72434e-01 1.08000e+02]
 [7.87837e-01 5.04500e-02 6.64494e-01 6.73860e-02 2.01000e+02]
 [5.42881e-01 1.27414e-01 9.72143e-01 4.86363e-01 1.81000e+02]
 [7.41424e-01 7.03198e-01 1.13026e-01 8.03548e-01 1.37000e+02]
 [8.65406e-01 3.01960e-02 3.72710e-02 1.07926e-01 1.89000e+02]
 [8.48243e-01 2.40763e-01 2.63083e-01 5.70242e-01 1.82000e+02]
 [6.08890e-02 6.20750e-02 1.24436e-01 4.03780e-02 1.96000e+02]
 [7.41341e-01 1.18998e-01 6.38528e-01 4.07312e-01 2.050

# Optimal Values for the variables

In [ ]:
ga = .8
gb = .51
gd = .37
gc = .01
print(ga,gb,gc,gd)

0.8 0.51 0.01 0.37


### Rough

In [ ]:
sp = np.linspace(0,1,2) # change last parameter for fine tuning
gco = 0 # global count
# ga,gb,gc = 0,0,0 # final parameter values

# a = 0.81
# b = 0.91
# c = 0.57
a = .89
b = .966
c = .50
d = 0.03
# if a==0 and b==0 and c==0:
    # continue
tc = 0 # temp count
k =  5
count = 0
de = {}
for i in range(Q1.shape[0]): # iterate all duplicate qns
    te = []
    for j in range(Q2.shape[0]): # iterate over available qns
        te.append(ga*SC[j][i][0]+gb*SC[j][i][1]+gc*SC[j][i][2] + gd*SC[j][i][3]) # composer score
        de[j] = te[j]

    x = te.index(max(te)) # get index of maximum element
    if Q2.loc[x,'OId'] == df.loc[i,'OId']: # if correct original qn is detected
        tc += 1
    temp_count= 0
    for key, val in sorted(de.items(), key= lambda item: item[1], reverse=True):
      if (temp_count<k):
        if Q2.loc[key,'OId'] == df.loc[i,'OId']:
          count+=1
          break
        temp_count+=1
      else:
        break

In [ ]:
print(count)

144


## Studying the effects of each component in decision making

In [27]:
tc = 0 # temp count
k =  20
count = [0,0,0] # 5,10,20
ga,gb,gc = 1, 1, 1 # normalised
# print(ga,gb,gc,gd)
original_data = df[['OId','OTitle','OBody','OTags']].loc[300:1500-1].copy() # original qns
test_data = df[['DId','DTitle','DBody','DTags']].loc[300:1500-1].copy() # doop qns
for i in range(300,300+test_data.shape[0]): # iterate all duplicate qns
    de = {}
    te = []
    # print(test_data.loc[i,'DTags'])
    x = TokStem(hbt(test_data.loc[i,'DBody']),0)
    y = rmTags(test_data.loc[i,'DTags']).split()
    z = TokStem(hbt(test_data.loc[i,'DTitle']),0)
    
    # text = (x+ y)

    # corp = id2word.doc2bow(text)

    # wa, wb = np.array(lda_model[corp])[:,1], lda_train[i]
        
    b= ['OTitle','OBody','OTags']
    for j in range(Q2.shape[0]): # iterate over available qns
    #     wa, wb = np.array(lda_model[corp])[:,1], lda_train[j] # lda train has Q1 qns and then Q2 qns
    #     topic_similarity = np.dot(wa,wb)/(np.linalg.norm(wa)*np.linalg.norm(wb))# TopicSC
    #     if (np.isnan(topic_similarity)):
    #       topic_similarity = 0
        # te.append(ga*cos_sim(z,Q2.loc[j,b[0]].split()) + gb*cos_sim(x,Q2.loc[j,b[1]].split())+ gc*cos_sim(y,Q2.loc[j,b[2]].split()) + gd*topic_similarity) # composer score
        te.append(cos_sim(y,Q2.loc[j,b[2]].split())) # composer score
        de[j] = te[j]

    # x = te.index(max(te)) # get index of maximum element
    # if Q2.loc[x,'OId'] == df.loc[i,'OId']: # if correct original qn is detected
    #     tc += 1
    temp_count= 0
    iter_temp = sorted(de.items(), key= lambda item: item[1], reverse=True)
    ### For k = 5
    k = 5
    for key, val in iter_temp:
      if (temp_count<k):
        if Q2.loc[key,'OId'] == df.loc[i,'OId']:
          count[0]+=1
          break
        temp_count+=1
      else:
        break
    # print("R@5 - ",count[0])
    ###m For k = 10
    temp_count = 0
    k = 10
    for key, val in iter_temp:
      if (temp_count<k):
        if Q2.loc[key,'OId'] == df.loc[i,'OId']:
          count[1]+=1
          break
        temp_count+=1
      else:
        break
    # print("R@10 : ",count[1])
     ###m For k = 20
    k = 20
    temp_count = 0
    for key, val in iter_temp:
      if (temp_count<k):
        if Q2.loc[key,'OId'] == df.loc[i,'OId']:
          count[2]+=1
          break
        temp_count+=1
      else:
        break
    # print("R@20 :- ",count[2])
print("Recall-Rate@5= ", count[0]/1200)
print("Recall-Rate@10= ", count[1]/1200)
print("Recall-Rate@20= ", count[2]/1200)


Recall-Rate@5=  0.26166666666666666
Recall-Rate@10=  0.33166666666666667
Recall-Rate@20=  0.3983333333333333


### For Title only analyser: (10min)

Recall-Rate@5=  0.30666666666666664

Recall-Rate@10=  0.375

Recall-Rate@20=  0.44583333333333336

### For Body only analyser: (1hr)

Recall-Rate@5=  0.2525

Recall-Rate@10=  0.30666666666666664

Recall-Rate@20=  0.37666666666666665

### For Tag only analyser: (4min)

Recall-Rate@5=  0.26166666666666666

Recall-Rate@10=  0.33166666666666667

Recall-Rate@20=  0.3983333333333333




# Return top 'k' questions

In [43]:
def top_k_qn(title,body,tags,k):
    tc = 0 # temp count
#     k =  5 ###### select the k value required
    count = 0
    de = {}
    ga,gb,gc,gd = 0.8,0.51,0.37,0.01
    print(ga,gb,gc)
    fs = '' ## string to store the output serial stream
    te = []
    # print(test_data.loc[i,'DTags'])
    x = TokStem(hbt(title),0)
    y = rmTags(body).split()
    z = TokStem(hbt(tags),0)

    text = (x+ y)
    corp = id2word.doc2bow(text)
    wa, wb = np.array(lda_model[corp])[:,1], lda_train[i]

    b= ['OTitle','OBody','OTags']
    temp_count= 0

    for j in range(Q2.shape[0]): # iterate over available qns
        wa, wb = np.array(lda_model[corp])[:,1], lda_train[j] # lda train has Q1 qns and then Q2 qns
        topic_similarity = np.dot(wa,wb)/(np.linalg.norm(wa)*np.linalg.norm(wb))# TopicSC
        if (np.isnan(topic_similarity)):
          topic_similarity = 0
        te.append(ga*cos_sim(z,Q2.loc[j,b[0]].split()) + gb*cos_sim(x,Q2.loc[j,b[1]].split())+ gc*cos_sim(y,Q2.loc[j,b[2]].split()) + gd*topic_similarity) # composer score
        de[j] = te[j]
    # x = te.index(max(te)) # get index of maximum element
    # if Q2.loc[x,'OId'] == df.loc[i,'OId']: # if correct original qn is detected
    #     tc += 1

    for key, val in sorted(de.items(), key= lambda item: item[1], reverse=True):
      # if temp_count == 0:
      #   temp_count += 1
      #   continue
#       print(" ----- Question ---- ", temp_count)
      fs += "\n \n ----- Question " + str(temp_count) + "----\n"
      fs += "\n Score---> " +  str(round(val,4))
#       print("Title --->", df.loc[key,'OTitle'])
      fs +=  "\n Title ---> " + str(df.loc[key,'OTitle'])
#       print("Question --->", df.loc[key,'OBody'])
      fs += "\n Body ---> " + str(df.loc[key,'OBody'])
#       print("Tags --->", df.loc[key,'OTags'])
      fs += "\n Tags ---> " + str(df.loc[key,'OTags'])
      # print(Q2.loc[key], df.loc[no])
      fs += "\n---------------------------------------------------------------"
      if (temp_count<k):
        temp_count+=1
      else:
        break
    return fs
#       print(fs)

# top_k_qn("Choosing a database type", "<p>When would you use a bigtabe/simpledb database vs a Relational database?</p>","<architecture><relational><amazon-simpledb><bigtable>")

In [31]:
######### import the required libraries
import tkinter as tk
from tkinter import *
from tkinter import ttk
from tkinter import filedialog as fd
from tkinter.messagebox import showinfo
from PIL import ImageTk,Image
import time 
import matplotlib.pyplot as plt
import tkinter.scrolledtext as scrolledtext

In [56]:


# !apt-get install -y xvfb # Install X Virtual Frame Buffer
# import os
# os.system('Xvfb :1 -screen 0 1600x1200x16  &')    # create virtual display with size 1600x1200 and 16 bit color. Color can be changed to 24 or 8
# os.environ['DISPLAY']=':1.0'    # tell X clients to use our virtual DISPLAY :1.0

######### create the root window
root = tk.Tk()
root.title('Similar Question detector')
root.resizable(True, True)
root.geometry('1920x1080')
#root.config(bg='yellow')

my_frame = Frame(root, width=576, height=324,bg='black')
my_frame.pack(fill="both", expand=True)

img = ImageTk.PhotoImage(file="bg.jpg")
label = Label(
    my_frame,
    image=img
)
label.place(x=0, y=0)

####################### global &/ initialisations
title,body,tag = "Title","Body","Tag" # Entries to be entered by the user
titles = ["one","two","three"]
kval = 5

#######################

def on_change1(tite): # retrieve the title entered
    global title,titles
    title = tite.widget.get()
    print("Title Entered:",title)

def on_change2(bode): # retrieve the body entered
    global body
    body = bode.widget.get()
    print("Body Entered:",body)

def on_change3(tage): # retrieve the tags entered
    global tag
    tag = tage.widget.get()
    print("Tags Entered:",tag)
    
def on_change4(ke): # retrieve the k-value entered
    global kval
    kval = int(ke.widget.get())
    print("K-value Entered:",kval)

def sub():
    global title, body, tag
    fs = top_k_qn(title,body,tag,kval)
    txt.delete('1.0', END)
    txt.insert('insert', fs,"\n")

    

########################

# entry for title =====
L2 = Label(root, text="Enter the Title:",fg='black',bg='white')
L2.pack( side = LEFT)
L2.place( x = 50,y=50)
tite=Entry(root, bd=2,textvariable=title)
tite.pack()
tite.place(x=180, y=45,width = 1600, height=30)
tite.bind("<Return>",on_change1)

# entry for body =====
L3 = Label(root, text="Enter the Body:",fg='black',bg='white')
L3.pack( side = LEFT)
L3.place(x=50,y=90)
bode=Entry(root, bd=2,textvariable=body)
bode.pack()
bode.place(x=180, y=85,width = 1600, height=30)
bode.bind("<Return>",on_change2)

# entry for tags =====
L4 = Label(root, text="Enter the Tags:",fg='black',bg='white')
L4.pack( side = LEFT)
L4.place( x = 50,y=130)
tage=Entry(root, bd=2,textvariable=tag)
tage.pack()
tage.place(x=180, y=125,width = 1600, height=30)
tage.bind("<Return>",on_change3)

# entry for k value =====
L5 = Label(root, text="Enter k-value:",fg='black',bg='white')
L5.pack( side = LEFT)
L5.place( x = 50,y=170)
ke=Entry(root, bd=2,textvariable=kval)
ke.pack()
ke.place(x=180, y=165,width = 100, height=30)
ke.bind("<Return>",on_change4)

#####################

# open button 1 ======> submit button
open_button_1 = Button(root,bg='black', fg='white', text='Submit',command=sub)
open_button_1.pack(expand=True)
open_button_1.place(x=950, y=200) #1238,400

txt = scrolledtext.ScrolledText(root, undo=True, wrap='word',width=150,height=40)
txt['font'] = ('consolas', '12')
txt.pack(expand=True, fill='both')
txt.place(x=200, y=250)

########################

# run the application
root.mainloop()


Title Entered: title database
Body Entered: body database
Tags Entered: tags database
K-value Entered: 10
0.8 0.51 0.37


<ipython-input-11-d24be59fa1d4>:37: RuntimeWarning: invalid value encountered in double_scalars
  ans = np.dot(wa,wb)/(np.linalg.norm(wa)*np.linalg.norm(wb))
